# Research Question and Background

As a fundamental pillar of culture, stories are a reflection of culture, a product of time and place. Horror stories, then, are a reflection of a societal fears, and the boundaries they draw between good and evil, health and perversity, us and them are a reflection of a society’s social state. <br>
<br>
Current popular science fiction stories center around fears of artificial intelligence and the boundaries between human and robot. This fear is rooted in the larger tradition of horror, science fiction, and Gothic stories of piecing together a body from deviance (race, class, gender, sexuality, or technology) and is heavily tied to the production of knowledge.<br>
<br>
Similar to how developments in artificial intelligence encourage creative writers to engage with social, moral, and future implications of technology, Mary Shelley's classic *Frankenstein* was written in context of recent scientific developments. Luigi Galvani had found that frog legs twitched with electric shocks, and Giovanni Aldini found that a recently hanged man's muscles twitched with an electric shock [publicdomain]. Additionally, the Englightment's central subject of the material versus the spiritual is explored through Frankenstein's monster [wedding night]. <br>
<br>
In *Discipline and Punish*, Michel Foucault analyzes how the Enlightenment's focus on the material versus the spiritual shifted the focus of discipline from the body to the soul. Crime literature followed, moving from the confession and gallows speech to detective fiction. In Gothic literature, as Judith Halberstam argues in *Skin Shows*, crime is embodied within the deviant form of the monster [skin shows]. Halberstam argues that the shift in media from literature to film pushed horror narrow its scope from deviant race, class, and gender to deviant sexuality and gender. Since film's depiction of a monster will always fall short to the reader's imagination, film depends on the explicit violation of female bodies rather than the sight of the monster [skin shows]. She uses *The Silence of the Lambs* to demonstrate how Buffalo Bill's identity crisis is a category crisis of sexual identity. However, this shift is also pushed by the Enlightenment's shift from the body to the soul, pushing horror from physical monstrosity to psychological and cultural monstrosity. Thus, her argument extends to modern horror literature, such as the Hannibal Lecter series, the literary basis for *The Silence of the Lambs*. <br>
<br>
In addition the category crisis of sexuality and gender, modern technological advancements have created category crisis between human and robot. [AAAAAAAAAAAA] <br>
<br>
The tradition of analyzing horror stories from a qualitative standpoint (psychological, historical, philosophical) is long – here, I approach this argument computationally.

# Method

To analyze trends in speculative fiction, I use data scraped by Carnegie Mellon University. The CMU book dataset is comprised of 16.5K books, with information such as author, published date, and summary from Wikipedia. I consider horror, science fiction, and gothic genres to be of interest in examining fears represented through literature. For a full list of genres included in the analysis, refer to appendix A. <br>
<br>
Using a common method for computational text analysis, term frequency-inverse document frequency (TF-IDF), I identify the relevance of topics (monster, robot) within the summary of a book. TF-IDF is a numerical measure for how relevant a word is to a document, and it accounts for how often a word appears across documents. Thus, since words like "and" appear commonly in many documents, it is deemed less important; while words like "monster" appear less commonly overall, but a document that mentions it often is quite likely about monsters. The summaries were preprocessed using standard methods -- removal of stopwords (e.g. "the"), lemmatization (reducing words to their root, e.g. "running" to "run"), and bigramization (freezing words pairs that appear frequently, e.g. "scary monster" to "scary_monster"). <br>
<br>
Since a majority of the books in the dataset were published recently, there is a baseline growth in relevance of most words. The one-hundred most 'relevant' words in the entire dataset were used to approximate this baseline growth. To get an accurate understanding of the change in relevance of topics of interst, 
and the baseline was subtracted from the change in relevance of monsters and robots.


# Relevance of Tropes

# Context

# Conclusion

# Appendix

A. Genres included in "speculative fiction": horror, gothic, science fiction, alien, biopunk, cyberpunk, ghost, paranormal, science fantasy, speculative fiction, steampunk, supernatural, time travel, techno, time travel, vampire, zombie.

# References

[shelley-galvanism]: 
https://publicdomainreview.org/essay/the-science-of-life-and-death-in-mary-shelleys-frankenstein>.<br>
[wedding night]: https://www.jstor.org/stable/pdf/778795.pdf?refreqid=excelsior%3Afb07bf6df1d12df4b85bc509a3a7bdca&ab_segments=&origin=&initiator=